<a href="https://colab.research.google.com/github/danh1uc4/PythonDataScienceHandbook/blob/master/Coursera_GradeBook_Toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RUN FIRST (PACKAGE IMPORTS)

In [ ]:
import csv
import zipfile
import pandas as pd
import numpy as np
from google.colab import files


import gspread
from google.auth import default
from google.colab import auth


# MOOC Grades


## **1. Enter the number of MOOCs.**

* If you are only uploading MOOC 1 or the course only has one MOOC, enter 1.
* To upload both MOOCs, change the number to 2.

In [ ]:
MOOC_Number = 1

## **2.   Download the gradebook from Canvas and upload the file below.**

In [ ]:
moocGradesCanvas_uploaded = files.upload()
moocGradesCanvas_filename = list(moocGradesCanvas_uploaded.keys())[0]

Saving 2025-09-08T2316_Grades-mba_549_120258_250174.csv to 2025-09-08T2316_Grades-mba_549_120258_250174 (1).csv


## **3.   Export the gradebook from the private session on Coursera ([instruction](https://www.coursera.support/s/article/360023044471-Download-Gradebook-Exports-for-Groups-Private-Sessions-and-Degree-Programs?)) and Upload the gradebook file below. Please upload the zip file without extraction.**

MOOC 1 (or only one MOOC):

In [ ]:
moocGradesMOOC_uploaded = files.upload()
moocGradesMOOC_filename = list(moocGradesMOOC_uploaded.keys())[0]

Saving global_impact_cross_cultural_management_9amab_1757391574368.zip to global_impact_cross_cultural_management_9amab_1757391574368 (1).zip


MOOC 2 (optional)

In [ ]:
mooc2GradesMOOC_uploaded = files.upload()
mooc2GradesMOOC_filename = list(mooc2GradesMOOC_uploaded.keys())[0]

IndexError: list index out of range

## **4. Run the below code.**

In [ ]:
moocGradesCanvas = pd.read_csv(moocGradesCanvas_filename)
columns_to_keep = moocGradesCanvas.columns[:6].tolist()
columns_to_keep += [col for col in moocGradesCanvas.columns if "Coursera MOOC" in col]
columns_to_keep = [col for col in columns_to_keep if moocGradesCanvas.loc[1,col]!="(read only)"]
if MOOC_Number != 2:
  columns_to_keep = [col for col in columns_to_keep if "Coursera MOOC 2" not in col]
moocGradesCanvas = moocGradesCanvas[columns_to_keep]

columns_to_keep = ["External Student ID", "Email", "Course Passed", "Completed with CC"]


with zipfile.ZipFile(moocGradesMOOC_filename, 'r') as zip_ref:
    zip_ref.extractall()
moocGradesMOOC = pd.read_csv("gradebook.csv")
moocGradesMOOC = moocGradesMOOC[columns_to_keep]
moocGradesMOOC["NetId"] = moocGradesMOOC["Email"].str.split('@').str[0]

if MOOC_Number == 2:
  with zipfile.ZipFile(mooc2GradesMOOC_filename, 'r') as zip_ref:
    zip_ref.extractall()
  mooc2GradesMOOC = pd.read_csv("gradebook.csv")
  mooc2GradesMOOC = mooc2GradesMOOC[columns_to_keep]
  mooc2GradesMOOC["NetId"] = mooc2GradesMOOC["Email"].str.split('@').str[0]

for idx, row in moocGradesCanvas.iterrows():

    user_id = row["Integration ID"]
    if user_id in moocGradesMOOC["External Student ID"].values:
        score = moocGradesMOOC.loc[moocGradesMOOC["External Student ID"] == user_id, "Completed with CC"].values[0]
        for col in moocGradesCanvas.columns:
            if "Coursera MOOC" in col and "Coursera MOOC 2" not in col:
                moocGradesCanvas.at[idx, col] = score
        if MOOC_Number == 2:
            if user_id in mooc2GradesMOOC["External Student ID"].values:
              score2 = mooc2GradesMOOC.loc[mooc2GradesMOOC["External Student ID"] == user_id, "Completed with CC"].values
              for col in moocGradesCanvas.columns:
                if "Coursera MOOC 2" in col:
                    moocGradesCanvas.at[idx, col] = score2
            else:
              for col in moocGradesCanvas.columns:
                  if "Coursera MOOC 2" in col:
                      moocGradesCanvas.at[idx, col] = np.nan

moocGradesCanvas.to_csv('out.csv', index=False)
files.download('out.csv')


mooc_cols = [col for col in moocGradesCanvas.columns if "Coursera MOOC" in col]
moocGradesCanvas[moocGradesCanvas[mooc_cols].isnull().any(axis=1)].to_csv("missing_account.csv", index=False)
files.download('missing_account.csv')

condition = (moocGradesMOOC["Course Passed"] == True) & (moocGradesMOOC["Completed with CC"] == False)
if MOOC_Number == 2:
  for col in moocGradesCanvas.columns:
    if "Coursera MOOC 2" in col:
      condition2 = (mooc2GradesMOOC["Course Passed"] == True) & (mooc2GradesMOOC["Completed with CC"] == False)
      condition = condition | condition2

moocGradesMOOC[condition].to_csv("passed_no_certificate.csv", index=False)
files.download('passed_no_certificate.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Group Roster Creation
Test Files: [Group Roster from Canvas](https://drive.google.com/file/d/1TxguyieqguFrzc5enTpxTIBsf0kOJP2z/view?usp=sharing), [Group List from GGP](https://docs.google.com/spreadsheets/d/1M3TdTIejjDgvY2zBEked7-SRKzvPh3IE/edit?usp=sharing&ouid=109745297990275365576&rtpof=true&sd=true), [Test Roster page](https://canvas.illinois.edu/courses/41621/pages/roster-test/edit).

## **0. Enter the Course rubric**


* Download the group list that GGP sent to the CM group.
* Copy the course rubric as listed in the sheet name. Enter the rubric below.
* Make sure the course rubric is enclosed in double quotes in the code chunk below (i.e. "BADM 520" instead of BADM 520).<img src='https://drive.google.com/uc?id=1WH0sEVmg_LluDGH1AFGuPmSLo0P8ozMc'>

In [ ]:
courseRubric = "ACCY 502"

## **1. Upload the group list file received from GGP**

In [ ]:
groupList_uploaded = files.upload()
groupList_filename = list(groupList_uploaded.keys())[0]

## **2. Download group roster file from Canvas and upload the file**

On Canvas, go to People -> Project Groups, and click "+Import". In the pop-up page, select "Download Course Roster CSV".<img src='https://drive.google.com/uc?id=1-QjYPEcatvFb7VxtyOsq3YKyhOjjsjkP'>

In [ ]:
groupRosterCanvas_uploaded = files.upload()
groupRosterCanvas_filename = list(groupRosterCanvas_uploaded.keys())[0]

## **3. Rund the code below to create group import file, group roster page code, and no-group list**

The below code will produce three files:
*   **{courseRubric}_groupRosterCanvas.csv:** Import this file to Canvas from People -> Project Groups -> +Import.
*   **{courseRubric}_noGroup.csv:** This file contains a list of students without a group. Students in this list are either on Canvas but not in group list,  or in group list but not on Canvas. Add the list to the [No Group List](https://uofi.app.box.com/file/1827122181395?s=nnjo67e23mtsityg8wfd6ew861koeebu) master sheet so the SAS team can verify their status.
*  **{courseRubric}_roster.txt:** This file contains HTML code that will be used for the Course Groups page on Canvas.
 * First, open an empty page in your own sandbox, and switch to the html editor. <img src='https://drive.google.com/uc?id=1G6Z0YTUsXef5uCqBQ3HfJKLqhYPyuKkQ'>
 * Paste the code into the editor, and click the button again to switch back to the rich text editor.
 * Copy the content and paste it onto the Course Roster page.

In [ ]:
groupList = pd.read_excel(groupList_filename, sheet_name=courseRubric).dropna(how="all")
groupList["NetId"] = groupList["NetId"].str.lower()
groupRosterCanvas = pd.read_csv(groupRosterCanvas_filename)

for idx, row in groupRosterCanvas.iterrows():
    user_id = row["user_id"].lower()
    if user_id in groupList["NetId"].values:
        groupName = groupList.loc[groupList["NetId"] == user_id, "Group Name"].values[0]
        groupRosterCanvas.loc[idx,"group_name"]=str(groupName)


roster_not_in_group = groupRosterCanvas[~groupRosterCanvas["user_id"].str.lower().isin(groupList["NetId"].str.lower())][["user_id"]].rename(columns={"user_id": "NetId"})
roster_not_in_group["source"] = "In Canvas roster, not in group list"
group_not_in_roster = groupList[~groupList["NetId"].str.lower().isin(groupRosterCanvas["user_id"].str.lower())][['NetId']]
group_not_in_roster["source"] = "In group list, not in Canvas roster"
non_enrollment = pd.concat([roster_not_in_group, group_not_in_roster], ignore_index=True)
groupRosterCanvas = groupRosterCanvas.dropna(subset=["group_name"])

groupRosterCanvas.to_csv(f"{courseRubric}_groupRosterCanvas.csv", index=False)
files.download(f"{courseRubric}_groupRosterCanvas.csv")

non_enrollment.to_csv(f"{courseRubric}_noGroup.csv", index=False)
files.download(f"{courseRubric}_noGroup.csv")

# read profile data from google spreadsheet
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
# https://docs.google.com/spreadsheets/d/1aXRQlU-B21AjGp4_ZfAr3p7_Lxxz2Fm-ZYL42hdQRm0/edit?gid=858321946#gid=858321946
spreadsheet = gc.open_by_key('1aXRQlU-B21AjGp4_ZfAr3p7_Lxxz2Fm-ZYL42hdQRm0')
# Below is a copy of the profile form in my Google Drive for testing purpose
# spreadsheet = gc.open_by_key('1XUppv1dkE8cCoB80IaPp_qg19VrD-mPi0vGc1M4_EIk')
worksheet = spreadsheet.get_worksheet(0)
profileForm = pd.DataFrame(worksheet.get())
profileForm.columns = profileForm.iloc[0]
profileForm = profileForm.drop(0)

studentlist = groupList['NetId']
filtered_profile = profileForm[profileForm.iloc[:,3].isin(studentlist)].iloc[:, [3, -1]]
filtered_profile.columns=["user_id", "linkedin"]
filtered_profile = filtered_profile.drop_duplicates(subset='user_id', keep="last")

groupRosterCanvas["full_name"] = groupRosterCanvas["name"].apply(lambda x: ' '.join(x.split(', ')[::-1]) if pd.notna(x) else x)
groupRosterCanvas = groupRosterCanvas.merge(filtered_profile, on="user_id", how="left")
groupRosterCanvas = groupRosterCanvas.merge(groupList[['NetId','Illinois Email']],left_on='user_id', right_on='NetId', how='left')
if "Matching Times" in groupList:
  groupRosterCanvas = groupRosterCanvas.merge(groupList[['NetId','Matching Times']],left_on='user_id', right_on='NetId', how='left')

def missinglink(linkedin, full_name):
    fullName = f"{full_name}"
    if pd.isna(linkedin) or linkedin.strip() == "":
        return " "
    elif "https://" not in linkedin:
        return f'<a href="https://{linkedin}"><img src="http://i.imgur.com/Z9pDAGI.png" alt="LinkedIn profile for {fullName}" height="15px" width="15px" margin="1px" /></a> '
    else:
        return f'<a href="{linkedin}"><img src="http://i.imgur.com/Z9pDAGI.png" alt="LinkedIn profile for {fullName}" height="15px" width="15px" margin="1px" /></a> '


groups = groupRosterCanvas.groupby("group_name")


html_filename = f"{courseRubric}_roster.txt"
with open(html_filename, "w", encoding="utf-8") as outfile:
    for group_name, students in groups:
        outfile.write(f"<h2>Group {group_name}</h2>\n<ul>")
        if "Matching Times" in groupList:
          outfile.write(f"<p>Suggested Meeting Times: {groups.get_group(group_name)['Matching Times'].iloc[0]}</p>")
        for i, (_, student) in enumerate(students.iterrows()):
            full_name = student["full_name"]
            email = student["Illinois Email"]
            #email=student['user_id']+"@illinois.edu"
            linkedin = student["linkedin"] if "linkedin" in student else ""
            linkedin = missinglink(linkedin, full_name)
            nameline = f"{full_name} "
            # emailline = f'<a href="mailto:{email}">{email}</a>, \n'
            comma = ", " if i < len(students) - 1 else ""
            studenthtml = f"{full_name} {linkedin} <a href='mailto:{email}'>{email}</a>{comma} "
            outfile.write(f'<li style="display: inline;">{studenthtml}</li>\n')
        outfile.write("</ul>\n")

    # outfile.write("</body>\n</html>")

files.download(html_filename)


# Teammate Evaluation Comments